# Chapter 05 실습 2: 전이학습 — Flowers 분류

## 목표
EfficientNetV2B0 전이학습으로 꽃 이미지 5종을 분류한다.

In [ ]:
# 필수 라이브러리 임포트
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
import pathlib

# 한글 폰트 설정
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

# 재현성 시드
tf.random.set_seed(42)
np.random.seed(42)

print(f'TensorFlow 버전: {tf.__version__}')

# 하이퍼파라미터 설정
IMG_SIZE   = 224         # EfficientNetV2B0 입력 크기 (기본 224, 하지만 V2는 260도 권장)
BATCH_SIZE = 32
NUM_CLASSES = 5          # 꽃 5종 (daisy, dandelion, roses, sunflowers, tulips)
AUTOTUNE   = tf.data.AUTOTUNE

In [ ]:
# Flowers 데이터셋 다운로드 및 로드

# tf.keras.utils.get_file로 TensorFlow 공식 flowers 데이터셋 다운로드
dataset_url = 'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz'

data_dir = tf.keras.utils.get_file(
    fname='flower_photos',           # 저장될 파일/폴더명
    origin=dataset_url,
    untar=True,                      # tgz 파일 자동 압축 해제
    cache_subdir='datasets/flowers'  # ~/.keras/datasets/flowers/ 에 저장
)
data_dir = pathlib.Path(data_dir)

# 데이터셋 구조 확인
print(f'데이터 디렉토리: {data_dir}')
print(f'\n폴더 구조:')
total_images = 0
class_names = sorted([item.name for item in data_dir.glob('*') if item.is_dir()])
print(f'클래스 목록: {class_names}')
print()

for class_name in class_names:
    class_dir = data_dir / class_name
    n_images = len(list(class_dir.glob('*.jpg')))
    total_images += n_images
    print(f'  {class_name:<15}: {n_images:>5}개 이미지')

print(f'\n전체 이미지 수: {total_images:,}개')

# 클래스 이름 한국어 매핑
CLASS_KO = {
    'daisy':      '데이지',
    'dandelion':  '민들레',
    'roses':      '장미',
    'sunflowers': '해바라기',
    'tulips':     '튤립'
}

# image_dataset_from_directory로 데이터셋 생성
# 80% 학습, 20% 검증으로 분할
train_ds_raw = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='training',
    seed=42,
    image_size=(IMG_SIZE, IMG_SIZE),  # 자동 리사이즈
    batch_size=BATCH_SIZE,
    label_mode='int'                  # 정수 레이블
)

val_ds_raw = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='validation',
    seed=42,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode='int'
)

# 클래스 이름 확인
class_names = train_ds_raw.class_names
print(f'\n데이터셋 클래스 순서: {class_names}')
print(f'학습 배치 수: {len(train_ds_raw)}')
print(f'검증 배치 수: {len(val_ds_raw)}')

In [ ]:
# 전처리 파이프라인 구성: 정규화 + 데이터 증강 + prefetch

# ===== 데이터 증강 레이어 정의 =====
# 학습 시에만 적용, 검증 시에는 사용 안 함
data_augmentation = tf.keras.Sequential([
    # 좌우 뒤집기 (꽃은 좌우 대칭)
    tf.keras.layers.RandomFlip('horizontal'),
    
    # 최대 10% 회전
    tf.keras.layers.RandomRotation(0.1),
    
    # 최대 10% 줌
    tf.keras.layers.RandomZoom(0.1),
    
    # 밝기 변화 (±20%)
    tf.keras.layers.RandomBrightness(0.2),
], name='data_augmentation')


def preprocess_train(images, labels):
    """
    학습용 전처리: 증강 + 정규화
    EfficientNetV2B0은 [0,255] 입력을 내부적으로 전처리하므로 (rescaling preprocessing 내장)
    별도 정규화 없이 그대로 전달 (내장 전처리 사용)
    """
    # 데이터 증강 적용 (학습 시에만)
    images = data_augmentation(images, training=True)
    # float32로 변환 (uint8 → float32)
    images = tf.cast(images, tf.float32)
    return images, labels


def preprocess_val(images, labels):
    """
    검증용 전처리: 정규화만 (증강 없음)
    """
    images = tf.cast(images, tf.float32)
    return images, labels


# 최종 학습/검증 데이터셋 파이프라인
train_ds = (
    train_ds_raw
    .map(preprocess_train, num_parallel_calls=AUTOTUNE)  # 병렬 전처리
    .shuffle(buffer_size=1000)                            # 무작위 셔플
    .prefetch(AUTOTUNE)                                   # 미리 불러오기 (GPU 병목 방지)
)

val_ds = (
    val_ds_raw
    .map(preprocess_val, num_parallel_calls=AUTOTUNE)
    .prefetch(AUTOTUNE)
)

print('데이터 파이프라인 구성 완료')
print(f'학습 파이프라인: map(증강+변환) → shuffle → prefetch')
print(f'검증 파이프라인: map(변환) → prefetch')
print()

# 증강된 이미지 샘플 시각화
sample_batch_images, sample_batch_labels = next(iter(train_ds_raw))
sample_image = sample_batch_images[0:1]  # 첫 번째 이미지만

fig, axes = plt.subplots(2, 5, figsize=(15, 6))

# 원본 이미지 5개
for i in range(5):
    axes[0, i].imshow(sample_batch_images[i].numpy().astype(np.uint8))
    class_name = class_names[sample_batch_labels[i].numpy()]
    axes[0, i].set_title(f'원본: {CLASS_KO.get(class_name, class_name)}')
    axes[0, i].axis('off')

# 동일 이미지에 증강 5번 적용
first_image = sample_batch_images[0:1]
first_label = sample_batch_labels[0]
for i in range(5):
    aug_img = data_augmentation(first_image, training=True)
    axes[1, i].imshow(aug_img[0].numpy().astype(np.uint8))
    axes[1, i].set_title(f'증강 #{i+1}')
    axes[1, i].axis('off')

plt.suptitle('데이터 증강 효과 (아래 행: 동일 이미지에 증강 반복 적용)', fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# Feature Extraction 단계: EfficientNetV2B0 + 새 분류 헤드

# EfficientNetV2B0 기본 모델 로드 (ImageNet 사전 학습)
base_model = tf.keras.applications.EfficientNetV2B0(
    include_top=False,                          # 분류 헤드 제거
    weights='imagenet',                          # ImageNet 가중치
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

# 기본 모델 완전 동결 (Feature Extraction 단계)
base_model.trainable = False

# ===== 모델 구성 =====
inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3), name='input_image')

# 기본 모델 통과 (BN은 추론 모드로 실행)
x = base_model(inputs, training=False)

# GlobalAveragePooling: (7, 7, 1280) → (1280,)
x = tf.keras.layers.GlobalAveragePooling2D(name='global_avg_pool')(x)

# 중간 Dense 레이어 (새로 학습할 레이어)
x = tf.keras.layers.Dense(256, activation='relu', name='dense_head')(x)

# Dropout으로 과적합 방지
x = tf.keras.layers.Dropout(0.4, name='dropout')(x)

# 최종 분류: 5개 꽃 클래스
outputs = tf.keras.layers.Dense(
    NUM_CLASSES, activation='softmax', name='flower_classifier'
)(x)

# 모델 생성
model = tf.keras.Model(inputs, outputs, name='Flowers_EfficientNetV2B0')

# 파라미터 통계
total_params = model.count_params()
trainable_params = sum([tf.size(w).numpy() for w in model.trainable_weights])
print(f'전체 파라미터:    {total_params:>10,}')
print(f'학습 가능 파라미터: {trainable_params:>10,}  (새 헤드만)')
print(f'동결된 파라미터:   {total_params-trainable_params:>10,}  (EfficientNetV2B0 본체)')
print(f'학습 효율:         {trainable_params/total_params*100:.2f}%')
print()

# Feature Extraction 컴파일
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Feature Extraction 학습
print('===== Feature Extraction 단계 학습 시작 =====')
history_fe = model.fit(
    train_ds,
    epochs=10,
    validation_data=val_ds,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_accuracy', patience=3,
            restore_best_weights=True, verbose=1
        )
    ],
    verbose=1
)

fe_val_acc = max(history_fe.history['val_accuracy'])
print(f'\nFeature Extraction 최고 검증 정확도: {fe_val_acc:.4f}')

In [ ]:
# Fine-Tuning 단계: 상위 20개 레이어 해동

print(f'EfficientNetV2B0 전체 레이어 수: {len(base_model.layers)}')
print()

# 기본 모델 해동
base_model.trainable = True

# 상위 20개 레이어만 학습 가능하게 설정
FINE_TUNE_FROM = len(base_model.layers) - 20

for layer in base_model.layers[:FINE_TUNE_FROM]:
    layer.trainable = False  # 하위 레이어 동결 유지

# 해동된 레이어 목록 출력
unfrozen_layers = [l.name for l in base_model.layers[FINE_TUNE_FROM:]]
print(f'해동된 레이어 ({len(unfrozen_layers)}개):')
for name in unfrozen_layers[-5:]:  # 마지막 5개만 표시
    print(f'  ... {name}')
print(f'  (이하 {len(unfrozen_layers)-5}개 레이어 포함)')
print()

# 파라미터 변화 확인
trainable_params_ft = sum([tf.size(w).numpy() for w in model.trainable_weights])
print(f'Fine-Tuning 학습 가능 파라미터: {trainable_params_ft:,}')
print(f'  (Feature Extraction 대비 {trainable_params_ft/trainable_params:.1f}배)')
print()

# Fine-Tuning 컴파일 (중요: 학습률 10배 낮춤)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),  # 1e-3 → 1e-4
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Fine-Tuning 학습 (Feature Extraction 이후 계속 학습)
fe_epochs = len(history_fe.history['loss'])

print('===== Fine-Tuning 단계 학습 시작 =====')
history_ft = model.fit(
    train_ds,
    epochs=fe_epochs + 10,
    initial_epoch=fe_epochs,       # Feature Extraction 에폭 이후부터
    validation_data=val_ds,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_accuracy', patience=4,
            restore_best_weights=True, verbose=1
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss', factor=0.5,
            patience=2, min_lr=1e-7, verbose=1
        )
    ],
    verbose=1
)

ft_val_acc = max(history_ft.history['val_accuracy'])
print(f'\nFine-Tuning 최고 검증 정확도: {ft_val_acc:.4f}')
print(f'정확도 향상: {fe_val_acc:.4f} → {ft_val_acc:.4f} '
      f'(+{(ft_val_acc - fe_val_acc)*100:.2f}%p)')

# 전체 학습 곡선 시각화
acc_all     = history_fe.history['accuracy']     + history_ft.history['accuracy']
val_acc_all = history_fe.history['val_accuracy'] + history_ft.history['val_accuracy']
loss_all    = history_fe.history['loss']         + history_ft.history['loss']
val_loss_all= history_fe.history['val_loss']     + history_ft.history['val_loss']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

for ax, train_vals, val_vals, ylabel, title in [
    (ax1, acc_all, val_acc_all, '정확도', '학습/검증 정확도'),
    (ax2, loss_all, val_loss_all, '손실', '학습/검증 손실')
]:
    ax.plot(train_vals, 'b-o', markersize=3, label='학습')
    ax.plot(val_vals,   'r-o', markersize=3, label='검증')
    ax.axvline(x=fe_epochs - 0.5, color='green', linestyle='--',
               label=f'Fine-Tuning 시작 (에폭 {fe_epochs})')
    ax.fill_betweenx([min(min(train_vals), min(val_vals)),
                       max(max(train_vals), max(val_vals))],
                      0, fe_epochs - 0.5, alpha=0.08, color='blue',
                      label='Feature Extraction')
    ax.fill_betweenx([min(min(train_vals), min(val_vals)),
                       max(max(train_vals), max(val_vals))],
                      fe_epochs - 0.5, len(acc_all), alpha=0.08, color='green',
                      label='Fine-Tuning')
    ax.set_xlabel('에폭')
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    ax.legend(fontsize=8)
    ax.grid(True, alpha=0.3)

plt.suptitle('Flowers 분류 전이학습 학습 곡선', fontsize=13)
plt.tight_layout()
plt.show()

In [ ]:
# 예측 시각화: 샘플 이미지 + 예측 레이블 + 신뢰도 막대 그래프

# 검증 데이터에서 샘플 배치 가져오기
val_images_batch, val_labels_batch = next(iter(val_ds_raw))

# 예측 수행
val_images_float = tf.cast(val_images_batch, tf.float32)
predictions = model.predict(val_images_float, verbose=0)
pred_labels = np.argmax(predictions, axis=1)
true_labels = val_labels_batch.numpy()

# 시각화 (10개 샘플)
n_show = min(10, BATCH_SIZE)
fig = plt.figure(figsize=(20, 8))

for idx in range(n_show):
    # 이미지 서브플롯
    ax_img = fig.add_subplot(2, n_show, idx + 1)
    ax_img.imshow(val_images_batch[idx].numpy().astype(np.uint8))
    
    true_cls  = class_names[true_labels[idx]]
    pred_cls  = class_names[pred_labels[idx]]
    true_ko   = CLASS_KO.get(true_cls, true_cls)
    pred_ko   = CLASS_KO.get(pred_cls, pred_cls)
    confidence = predictions[idx, pred_labels[idx]]
    
    # 정답이면 파란색, 오답이면 빨간색
    color = 'blue' if true_labels[idx] == pred_labels[idx] else 'red'
    ax_img.set_title(
        f'실제: {true_ko}\n예측: {pred_ko}\n신뢰도: {confidence:.2f}',
        fontsize=8, color=color
    )
    ax_img.axis('off')
    
    # 확률 분포 막대 그래프
    ax_bar = fig.add_subplot(2, n_show, n_show + idx + 1)
    bars = ax_bar.bar(
        range(NUM_CLASSES), predictions[idx],
        color=['#FF6B6B' if i == pred_labels[idx] else
               '#45B7D1' if i == true_labels[idx] else '#D3D3D3'
               for i in range(NUM_CLASSES)]
    )
    ax_bar.set_xticks(range(NUM_CLASSES))
    ax_bar.set_xticklabels(
        [CLASS_KO.get(cn, cn)[:3] for cn in class_names],
        fontsize=7, rotation=45
    )
    ax_bar.set_ylim([0, 1])
    ax_bar.set_ylabel('확률', fontsize=7)
    ax_bar.tick_params(axis='y', labelsize=7)

plt.suptitle(
    '예측 시각화 (파란 제목=정답, 빨간 제목=오답 / 막대: 빨강=예측, 하늘=정답)',
    fontsize=11
)
plt.tight_layout()
plt.show()

# 전체 검증 세트 정확도 계산
all_true, all_pred = [], []
for batch_images, batch_labels in val_ds_raw:
    batch_float = tf.cast(batch_images, tf.float32)
    batch_preds = model.predict(batch_float, verbose=0)
    all_true.extend(batch_labels.numpy())
    all_pred.extend(np.argmax(batch_preds, axis=1))

all_true = np.array(all_true)
all_pred = np.array(all_pred)
final_acc = np.mean(all_true == all_pred)

print(f'\n전체 검증 세트 정확도: {final_acc:.4f} ({final_acc*100:.2f}%)')

# 클래스별 정확도
print('\n클래스별 정확도:')
for i, cname in enumerate(class_names):
    mask = all_true == i
    cls_acc = np.mean(all_pred[mask] == all_true[mask])
    bar = '█' * int(cls_acc * 20) + '░' * (20 - int(cls_acc * 20))
    ko_name = CLASS_KO.get(cname, cname)
    print(f'  {ko_name:<6} {bar} {cls_acc:.4f}')

## 결과 분석 및 도전 과제

### 결과 분석

EfficientNetV2B0 전이학습을 통해 상대적으로 적은 데이터(약 3,670장)로도 높은 정확도를 달성할 수 있다.

| 단계 | 검증 정확도 | 학습 가능 파라미터 |
|------|-----------|------------------|
| Feature Extraction | ~85-88% | ~330K (헤드만) |
| Fine-Tuning (상위 20층) | ~90-93% | ~1.2M |

Fine-Tuning을 통해 Feature Extraction 대비 약 3~5%p 성능이 향상된다.

### 혼동 패턴 분석
- **민들레 ↔ 해바라기**: 노란색 꽃이라는 공통점으로 혼동 발생 가능
- **장미 ↔ 튤립**: 붉은색 계열로 혼동 가능

### 도전 과제

**과제 1: 다른 기본 모델과 비교**
```python
# EfficientNetB3, MobileNetV2, ResNet50 비교
models_to_try = [
    tf.keras.applications.EfficientNetB3,
    tf.keras.applications.MobileNetV2,
    tf.keras.applications.ResNet50
]
```

**과제 2: 데이터 증강 강도 실험**
- 증강 없음 vs 현재 vs 강한 증강 (`RandomContrast`, `RandomTranslation` 추가)
- 각 설정의 학습/검증 정확도 차이 분석

**과제 3: Fine-Tuning 해동 레이어 수 최적화**
```python
# 10개, 20개, 30개, 50개 레이어 해동 비교
for n_unfreeze in [10, 20, 30, 50]:
    # 각 설정으로 Fine-Tuning 후 성능 측정
    pass
```

**과제 4: 학습률 탐색 (LR Range Test)**
- 1e-6부터 1e-1까지 로그 스케일로 학습률을 변화시키며 손실 변화 관찰
- 손실이 가장 빠르게 감소하는 학습률 구간 찾기

**과제 5: 앙상블 (Ensemble)**
- Feature Extraction 모델과 Fine-Tuning 모델의 예측 평균
- 단일 모델 대비 성능 향상 여부 확인